In [ ]:
!poetry add optuna

### Sample : Optuna 개요

In [5]:
import optuna

In [ ]:
# Optuna Study 생성 -> blackbox Optimizer 및 관리 담당
study = optuna.create_study(directions="maximize")

# 최적화할 목적함수 및 시도 횟수, 조건 등을 주고 Optimize 수행
# 이때 각 시행별로 모델의 성능을 측정한 후 성능을 기반으로 다음 시도의 하이퍼파라미터를 탐색함
study.optimize(objective, n_trials=500) 

print(f"Best trial {study.best_trial}")

### Type

In [ ]:
# Categorical
activation = trial.suggest_categorical(name="activation",  choices=["ReLU", "ReLU6", "Hardswish"])

# Continuous
lr = trial.suggest_float(name="learning_rate", low=1e-5, high=1e-2)

# Integer
epochs = trial.suggest_int(name="epochs", low=10, high=30, step=10)

위의 다양한 타입의 하이퍼파라미터(최적해를 찾을 대상)는 아래와 같이 objective 함수에 정의된다.

In [43]:
import optuna

# objective 함수
def objective(trial):
    """Get objective score.

	Args:
		trial: optuna trial object
	
	Returns:
		float: Score value.
			Whether to maximize, minimize will determined in optuna study setup.
	"""
    sample = ["ReLU", "ReLU6", "Hardswish"]
    activation = trial.suggest_categorical("activation", sample)
    print(f"Suggested activation: {activation}")
    return sample.index(activation)
    
# 스터디 생성 및 최적화 실행
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)  

# 최적화된 결과 출력
print("Best activation:", study.best_params['activation'])

[I 2024-08-08 00:12:19,844] A new study created in memory with name: no-name-e131e5a3-cffc-421f-8958-fb79127d6091
[I 2024-08-08 00:12:19,846] Trial 0 finished with value: 2.0 and parameters: {'activation': 'Hardswish'}. Best is trial 0 with value: 2.0.
[I 2024-08-08 00:12:19,848] Trial 1 finished with value: 0.0 and parameters: {'activation': 'ReLU'}. Best is trial 1 with value: 0.0.
[I 2024-08-08 00:12:19,849] Trial 2 finished with value: 2.0 and parameters: {'activation': 'Hardswish'}. Best is trial 1 with value: 0.0.
[I 2024-08-08 00:12:19,850] Trial 3 finished with value: 0.0 and parameters: {'activation': 'ReLU'}. Best is trial 1 with value: 0.0.
[I 2024-08-08 00:12:19,851] Trial 4 finished with value: 2.0 and parameters: {'activation': 'Hardswish'}. Best is trial 1 with value: 0.0.


Suggested activation: Hardswish
Suggested activation: ReLU
Suggested activation: Hardswish
Suggested activation: ReLU
Suggested activation: Hardswish
Best activation: ReLU


`objective 함수`를 유연하게 꾸밀 수 있기 때문에 기존의 DL 프로세스에 optuna를 적용하기 어렵지 않다.

`if`문을 통한 `Conditional Type`역시 관리할 수 있다. 

In [ ]:
# 다음같이 objective를 분리하여 정의할 수도 있다.
def objective(trial)
    architecture = sample_model(trial)
    hyperparams = sample_hyperparam(trial)
    model = train_model(Model(architecture, verbose=True), hyperparams)
    score = evaluate(model)
return score

### Model 생성하기 : Model Config

모델을 코드상에 직접 구현하지 않고 yaml 파일 같은 Config로 관리하여 가져오기 위한 yaml 파일 생성

In [ ]:
# ResNet 50 예시

BACKBONE:
[
    [1, Conv, [64, 7, 2]],
    [1, Maxpool, [3, 2]],
    [3, BottleNeck, [64]],
    [4, BottleNeck, [128]],
    [6, BottleNeck, [256]],
    [3, BottleNeck, [512]],
    [1, GlobalAvgPool, []],
    [1, Flatten, []],
    [1, Linear, [1000, softmax]]
]


### Model 생성하기 : NAS의 접근법

NAS(Neural Architecture Search)는 좋은 모듈 Block(Macro)을 그대로 쓰되 micro한 부분을 탐색한다.

아래의 코드는 특정 분기에서 Macro는 유지하되 내부의 출력 채널, 커널 사이즈, 활성화 함수 등 micro한 부분을 변경하며 최적해를 찾는다.

In [ ]:
def objective(trial):
    n = trial.suggest_int("n_repeat", 1, 5)

    # Sample Normal Cell (NC) 
    model = []
    nc_args = [] 
    nc = trial.suggest_categoricall("normal_cell", ["Conv", "DWConv", "Bottleneck", "InvertedResidualv2", "MBConv"])

    if nc == "Conv":
        # Conv_args: [out_channel, kernel_size, stride, padding, activation] 
        out_channel = trial.suggest_int("normal_cell/out_channels", 16, 64)
        kernel_size = trial.suggest_int("normal_cell/kernel_size", 1, 7, step=2)
        activation = trial.suggest_categorical("normal_cell/activation", ["RELU", "RELU6", "Hardswish"])
        nc_args = [out_channel, kernel_size, 1, None, 1, activation] 
    elif nc == "DWConv":
        pass 
    elif nc == "Bottleneck":
        pass

    # Sample Reduction Cell(RC) 
    rc_args = [] 
    rc = trial.suggest_categorical("reduction_cell", ["InvertedResidualv2", "InvertedResidualv3", "MaxPool", "AvgPool"])

    if rc == "InvertedResidualv2":
        t = trial.suggest_int("reduction_cell/t", 1, 6)
        out_channel = trial.suggest_int("reduction_cell/out_channels", 16, 64)
        stride = 2
        rc_args = [t, out_channel, stride] 
    elif rc == "InvertedResidualv3":
        pass 
    elif rc == "MaxPool":
        pass

    model.append([n, nc, nc_args])
    model.append([1, rc, rc_args])
    model.append([n, nc, nc_args])
    model.append([1, rc, rc_args])
    model.append([n, nc, nc_args])
    model.append([1, "GlobalAvgPool", []])
    model.append([1, "Flatten", []])
    model.append([1, "Linear", [10]])

    model = model_generate(model)
    model = train(model)

    return evaluate(model)